# Intensional AMR-to-LF

This notebook runs through the intensional version of the translation function in section 3 of the paper.

In [123]:
import penman

## Free

A set `free` will store variables which take part in instance assignment.

In [124]:
free = set()

## World index

We use a count `world` that will track the world index.

When using this to index worlds in the lf we will do so with an underscore (i.e., **'w_1'**) to avoid naming clashes with variables which instantiate concepts like **'want-01'**.

In [125]:
world = 0

## Instance Assignment
Instance assignment adds the variable name to free. Uninstantiated variables are not added to free.

In [126]:
def instance_assignment(triple, free, world):
    lf = f'{triple[2]}({triple[0]})(w_{world})'
    free.add(triple[0])
    return lf, free

In [127]:
amr = '(x / P)'
g = penman.decode(amr).triples

instance_assignment(g[0], free, world)

('P(x)(w_0)', {'x'})

## Role Assignment
This rule is simpler than in the paper for two reasons:

- We do not need the lambda term in the simple case.
- We do not need a second rule for when the second argument is complex.

This is because we are working with `amr.triples` which specifies the first argument of the relation.

In [128]:
def role_assignment(triple, world):
    lf = f'{triple[1][1:]}({triple[0]})({triple[2]})(w_{world})'
    return lf

In [129]:
free = set()
amr = '(x / P :R y)'
g = penman.decode(amr).triples

role_assignment(g[1], world)

'R(x)(y)(w_0)'

## Close

`close()` takes a tuple of an lf and the set of instantiated free variables, and a world index.
- It existentially closes the free variables and removes them from the set.
- It introduces lambda abstraction over the world variables within the lf.

In [130]:
def close(lf, free, world):
    prefix = f'λw_{world}.∃' + ','.join(f'{x}' for x in free) + '.'
    free = set()
    lf = prefix + lf
    return lf, free

## Tree Depth

When computing the scope of **:content** we will need to reference the depth of the parent node.

We store this information in a dictionary to avoid computing it multiple times.

In [131]:
def node_depth(epidata):
    '''
    Uses penman epidata to determine node depth
    Returns dictionary {triple : depth} for triple in epidata
    where depth = depth of first node in triple
    '''

    depth = 0
    tree_depth = {}
    
    for triple in epidata:
        tree_depth[triple] = depth
        if epidata[triple]:
            for stack_op in epidata[triple]:
                if isinstance(stack_op, penman.layout.Push):
                    depth += 1
                elif isinstance(stack_op, penman.layout.Pop):
                    depth -= 1
    return tree_depth

## Translation function & Content Assignment

`translation_func()` and `cont_assignment()` are defined in the same cell as they call each other recursively.


- `translation_func()` iterates through the list of triples and calls either `instance_assignment()`, `role_assignment()` or `cont_assignment()`.

- `cont_assignment()` identifies the subgraph which forms the scope of **:content** and applies `translation_func()` to that subgraph.

In [132]:
def translation_func(graph, free, world, tree_depth):
    lf = ''
    triple_index = 0

    while triple_index < len(graph):
        triple = graph[triple_index]

        # instance assignment
        if triple[1] == ':instance':
            lf += ' & ' + instance_assignment(triple, free, world)[0]
            triple_index += 1
            
        # content assignment
        elif triple[1] == ':content':
            cont_lf, free, triple_index = cont_assignment(graph, free, world, tree_depth, triple_index)
            lf += ' & ' + cont_lf
            
        # role assignment
        else:
            lf += ' & ' + role_assignment(triple, world)
            triple_index += 1
            
    # Removes surplus leading conjunction
    lf = lf[3:]

    return lf, free


def cont_assignment(graph, free, world, tree_depth, triple_index):

    world += 1
    content_free = set()
    subgraph = []

    triple = graph[triple_index]
    triple_index += 1

    # build subgraph by iterating through remaining triples, if depth < or = that of triple, break
    for remaining_triple in graph[triple_index:]:
        if tree_depth[remaining_triple] <= tree_depth[triple]:
            break
        else: 
            subgraph.append(remaining_triple)
            triple_index += 1

    # Translate subgraph
    content_lf, content_free = translation_func(subgraph, content_free, world, tree_depth)

    content_lf = close(content_lf, content_free, world)[0]

    lf = f'cont({triple[0]})({content_lf})(w_{world-1})'

    free -= content_free

    return lf, free, triple_index

In [133]:
amr = '(x / P :R (y / Q :content (z / S :content (a / A))) :R2 z)'
g = penman.decode(amr)
tree_depth = node_depth(g.epidata)
graph = g.triples

cont_assignment(graph, set(), world, tree_depth, 3)[0]

'cont(y)(λw_1.∃z.S(z)(w_1) & cont(z)(λw_2.∃a.A(a)(w_2))(w_1))(w_0)'

## Examples

In [134]:
ex1 = '(a / admire-01 :ARG0 (b / boy) :ARG1 b))'
ex2 = '(b / believe-01 :ARG0 (b2 / boy) :content (s / sick-05 :ARG1 b2) :time (r / rain))'
ex3 = '(t / think-01 :ARG0 (b / boy) :content (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v /violin))))'

In [135]:
for ex in [ex1,ex2,ex3]:
    g = penman.decode(ex)
    tree_depth = node_depth(g.epidata)
    graph = g.triples
    free = set()
    world = 0
    
    lf, free = translation_func(graph, free, world, tree_depth)
    lf_closed = close(lf, free, world)[0]

    print(f'⟦{ex}⟧')
    print('=' + lf_closed)
    print('-' * 20)

⟦(a / admire-01 :ARG0 (b / boy) :ARG1 b))⟧
=λw_0.∃b,a.admire-01(a)(w_0) & ARG0(a)(b)(w_0) & boy(b)(w_0) & ARG1(a)(b)(w_0)
--------------------
⟦(b / believe-01 :ARG0 (b2 / boy) :content (s / sick-05 :ARG1 b2) :time (r / rain))⟧
=λw_0.∃b2,b,r.believe-01(b)(w_0) & ARG0(b)(b2)(w_0) & boy(b2)(w_0) & cont(b)(λw_1.∃s.sick-05(s)(w_1) & ARG1(s)(b2)(w_1))(w_0) & time(b)(r)(w_0) & rain(r)(w_0)
--------------------
⟦(t / think-01 :ARG0 (b / boy) :content (h / hope-01 :ARG0 (g / girl) :content (b2 / buy-01 :ARG0 g :ARG1 (v /violin))))⟧
=λw_0.∃t,b.think-01(t)(w_0) & ARG0(t)(b)(w_0) & boy(b)(w_0) & cont(t)(λw_1.∃g,h.hope-01(h)(w_1) & ARG0(h)(g)(w_1) & girl(g)(w_1) & cont(h)(λw_2.∃b2,v.buy-01(b2)(w_2) & ARG0(b2)(g)(w_2) & ARG1(b2)(v)(w_2) & violin(v)(w_2))(w_1))(w_0)
--------------------
